In [1]:
%%capture
%load_ext sql
%sql sqlite:///chinook.db

In [2]:
%%sql 
SELECT 
    name,
    type 
FROM sqlite_master
WHERE type IN ("table", "view");

 * sqlite:///chinook.db
Done.


name,type
album,table
artist,table
customer,table
employee,table
genre,table
invoice,table
invoice_line,table
media_type,table
playlist,table
playlist_track,table


## Which artist that we need to boost the sales first? 

In [3]:
%%sql 
SELECT *
FROM invoice_line
limit 5;

 * sqlite:///chinook.db
Done.


invoice_line_id,invoice_id,track_id,unit_price,quantity
1,1,1158,0.99,1
2,1,1159,0.99,1
3,1,1160,0.99,1
4,1,1161,0.99,1
5,1,1162,0.99,1


In [4]:
%%sql
WITH 
    cty as 
        (
        SELECT invoice_id
        FROM invoice 
        WHERE billing_country = "USA"   
        ),
    track_cty as 
        (
        SELECT il.track_id, sum(il.quantity) q
        FROM cty
        INNER JOIN  invoice_line il ON cty.invoice_id = il.invoice_id
        GROUP BY 1
        ),
    genr AS 
        (
        SELECT tr.genre_id, sum(t.q) tracks
        FROM track tr
        INNER JOIN track_cty t ON t.track_id = tr.track_id
        GROUP BY 1) 
        
SELECT g.name genre, genr.tracks total_tracks, CAST(genr.tracks AS FLOAT)/ (SELECT SUM(tracks)FROM genr) perc
FROM genr
INNER JOIN genre g ON g.genre_id = genr.genre_id
ORDER BY 2 DESC
LIMIT 10;

 * sqlite:///chinook.db
Done.


genre,total_tracks,perc
Rock,561,0.5337773549000951
Alternative & Punk,130,0.12369172216936251
Metal,124,0.11798287345385347
R&B/Soul,53,0.05042816365366318
Blues,36,0.03425309229305423
Alternative,35,0.03330161750713606
Latin,22,0.02093244529019981
Pop,22,0.02093244529019981
Hip Hop/Rap,20,0.019029495718363463
Jazz,14,0.013320647002854425


According to this, the artist that we need to buy first are Red Tone - Punk, Slim Jim Bites-Blues, Meteor and the Girls - Pop. This is because the past historical sales for those genre speicifically in US are higher than hip hop 

## EMPLOYEE SUPPORT PERFORMANCE

In [5]:
%%sql 
WITH 
    cst AS 
        ( 
        SELECT customer_id, SUM(total)sales
        from invoice
        GROUP BY 1),
    sprt_rep AS 
        (
        SELECT c.support_rep_id support_id, SUM(cst.sales) sales_performance
        FROM customer c
        INNER JOIN cst ON cst.customer_id = c.customer_id
        GROUP BY 1)
SELECT  
    e.first_name || " " || e.last_name employee_name, 
    e.hire_date,
    sp.sales_performance
FROM employee e
INNER JOIN sprt_rep sp ON sp.support_id = e.employee_id

 * sqlite:///chinook.db
Done.


employee_name,hire_date,sales_performance
Jane Peacock,2017-04-01 00:00:00,1731.5099999999998
Margaret Park,2017-05-03 00:00:00,1584.0000000000002
Steve Johnson,2017-10-17 00:00:00,1393.92


For the employee performance, Jane Peacock surely performing better than the others. 
However, looking at the hire_date, Jane Peacok was hired first before the others therefore, we still doesnt know which one is better, but steve, is actually better, this is because seeing from the latest hired person, he is almost catching up with Margaret Park.

## Total Purchases per Country 

In [6]:
%%sql
    WITH 
        cty_performance AS 
            (
            SELECT 
                c.country cty, 
                COUNT(DISTINCT c.customer_id) num_cust,
                SUM(i.total) sales,
                SUM(i.total) / COUNT(DISTINCT c.customer_id) average_value,
                AVG(i.total) average_order      
            FROM customer c
            INNER JOIN Invoice i ON i.customer_id = c.customer_id
            GROUP BY 1
            ),
        categorical_country AS 
            ( 
            SELECT 
                CASE 
                    WHEN num_cust = 1 then "Others"
                    ELSE cty
                END AS country, 
                ROUND(SUM(num_cust),2) total_customer,
                ROUND(sum(sales),2) total_sales,
                ROUND(AVG(average_value),2) average_sales_value,
                ROUND(AVG(average_order),2) average_order_value
            FROM cty_performance
            GROUP BY 1 
            )
SELECT 
    country,
    total_customer,
    total_sales,
    average_order_value,
    average_sales_value
    
FROM 
    (SELECT 
         cc.*, 
         CASE
            WHEN cc.country = "Others" THEN 0
            ELSE 1
            END AS sort
     FROM categorical_country cc
    )
ORDER BY sort DESC, total_sales DESC;

 * sqlite:///chinook.db
Done.


country,total_customer,total_sales,average_order_value,average_sales_value
USA,13.0,1040.49,7.94,80.04
Canada,8.0,535.59,7.05,66.95
Brazil,5.0,427.68,7.01,85.54
France,5.0,389.07,7.78,77.81
Germany,4.0,334.62,8.16,83.66
Czech Republic,2.0,273.24,9.11,136.62
United Kingdom,3.0,245.52,8.77,81.84
Portugal,2.0,185.13,6.38,92.57
India,2.0,183.15,8.72,91.58
Others,15.0,1094.94,7.45,73.0


It can be found that USA has the highest sales but the highest order value is in Czech republic

## DATA FOR THE PURCHASED ALBUM COMPARE WITH TRACKS 

In [7]:
%%sql 

WITH 
    album_tracks AS 
        (
        SELECT a.album_id, t.track_id
        FROM track t
        INNER JOIN album a on a.album_id = t.album_id)
SELECT *
FROM album_tracks 
ORDER BY 1 ASC
LIMIT 20;

 * sqlite:///chinook.db
Done.


album_id,track_id
1,1
1,6
1,7
1,8
1,9
1,10
1,11
1,12
1,13
1,14


In [8]:
%%sql 

WITH 
    album_tracks AS 
        (
        SELECT a.album_id, t.track_id tracks
        FROM track t
        INNER JOIN album a on a.album_id = t.album_id
        ),
    invoice_tracks AS 
        (
        SELECT invoice_id, track_id tracks
        FROM invoice_line
        ),
    combi AS 
        (
        SELECT *
        FROM invoice_tracks it
        INNER JOIN album_tracks at ON at.tracks = it.tracks
        ),
    inv_id_per_count AS 
        (
        SELECT invoice_id, album_id, COUNT(tracks) tracks
        FROM combi
        GROUP BY 1,2
        ),
    num_of_tracks AS
        (
        SELECT album_id, count(tracks) album_tracks
        FROM album_tracks
        GROUP BY 1
        ),
    comparison AS
        (
        SELECT *
        FROM inv_id_per_count id
        INNER JOIN num_of_tracks nt ON nt.album_id = id.album_id
        ),
    album_purchase AS 
        (
        SELECT 
            invoice_id,
            CASE 
                WHEN tracks = album_tracks THEN 1
                ELSE 0
            END AS album_purchase
        FROM comparison 
        ),
    final_product AS 
        (
        SELECT 
        invoice_id,
        CASE 
             WHEN MIN(album_purchase) = 1 THEN "yes"
             ELSE "no" 
        END AS album_buyers
        FROM album_purchase
        GROUP BY 1
        )
SELECT 
    album_buyers, 
    COUNT(invoice_id) number_of_invoice, 
    COUNT(invoice_id) / CAST((SELECT COUNT(DISTINCT invoice_id) from inv_id_per_count) AS FLOAT) perc 
FROM final_product
GROUP BY 1;
        
    

 * sqlite:///chinook.db
Done.


album_buyers,number_of_invoice,perc
no,500,0.8143322475570033
yes,114,0.18566775244299674


This can be concluded that the more of the people buy individual tracks instead of album, so that after considering the edges, it can be said that it is better to buy the most popular tracks instead of buying all album

In [9]:
%%sql

WITH invoice_first_track AS
    (
     SELECT
         il.invoice_id invoice_id,
         MIN(il.track_id) first_track_id
     FROM invoice_line il
     GROUP BY 1
    )

SELECT
    album_purchase,
    COUNT(invoice_id) number_of_invoices,
    CAST(count(invoice_id) AS FLOAT) / (
                                         SELECT COUNT(*) FROM invoice
                                      ) percent
FROM
    (
    SELECT
        ifs.*,
        CASE
            WHEN
                 (
                  SELECT t.track_id FROM track t
                  WHERE t.album_id = (
                                      SELECT t2.album_id FROM track t2
                                      WHERE t2.track_id = ifs.first_track_id
                                     ) 

                  EXCEPT 

                  SELECT il2.track_id FROM invoice_line il2
                  WHERE il2.invoice_id = ifs.invoice_id
                 ) IS NULL
             AND
                 (
                  SELECT il2.track_id FROM invoice_line il2
                  WHERE il2.invoice_id = ifs.invoice_id

                  EXCEPT 

                  SELECT t.track_id FROM track t
                  WHERE t.album_id = (
                                      SELECT t2.album_id FROM track t2
                                      WHERE t2.track_id = ifs.first_track_id
                                     ) 
                 ) IS NULL
             THEN "yes"
             ELSE "no"
         END AS "album_purchase"
     FROM invoice_first_track ifs
    )
GROUP BY album_purchase;

 * sqlite:///chinook.db
Done.


album_purchase,number_of_invoices,percent
no,614,1.0


In [13]:
%%sql

WITH invoice_first_track AS
    (
     SELECT
         il.invoice_id invoice_id,
         MIN(il.track_id) first_track_id
     FROM invoice_line il
     GROUP BY 1
    )
SELECT 
    (SELECT il2.track_id FROM invoice_line il2
    WHERE il2.invoice_id = ifs.invoice_id)
FROM invoice_first_track ifs

 * sqlite:///chinook.db
Done.


(SELECT il2.track_id FROM invoice_line il2 WHERE il2.invoice_id = ifs.invoice_id)
1158
3476
2516
3448
1986
1045
3477
2231
206
2663


In [14]:
%%sql
SELECT track_id from invoice_line
LIMIT 700;

 * sqlite:///chinook.db
Done.


track_id
1
1
1
1
1
1
1
1
2
2
